In [ ]:
pip install transformers pandas

In [ ]:
from transformers import pipeline
import pandas as pd

# Load the sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis")

# Read the dataset

data = pd.read_csv("/content/textmining cleaned.csv", encoding='latin-1')
# Extract the 'University name' and 'Generative AI statement text' columns
universities = data['University name'].tolist()
texts = data['Generative AI statement text'].tolist()

# Initialize a dictionary to store unique university sentiments
university_sentiments = {}

# Process each text for sentiment analysis (split long sequences)
for university, text in zip(universities, texts):
    if university not in university_sentiments:
        if len(text) > 512:  # If the text is longer than the model's limit
            # Split the long text into segments of 512 tokens or less
            segments = [text[i:i + 512] for i in range(0, len(text), 512)]
            # Perform sentiment analysis on each segment
            segment_sentiments = [sentiment_analysis(segment) for segment in segments]
            # Store the sentiment label with the university
            university_sentiments[university] = [sent['label'] for sublist in segment_sentiments for sent in sublist]
        else:
            # Process shorter texts directly
            sent = sentiment_analysis(text)
            university_sentiments[university] = [sent[0]['label']]

# Convert the dictionary into a DataFrame
output_df = pd.DataFrame([(key, value[0]) for key, value in university_sentiments.items()], columns=['University name', 'Sentiment'])

# Display the DataFrame
print(output_df)
